
# Künstlicher Assistent zur Annotation von Märchen

### Google Colab Arbeitplatz vorbereiten

In [ ]:
!git clone https://github.com/edadunashvili/Monographie.git

In [ ]:
cd /content/Monographie

In [ ]:
!pip install scikit-learn==1.1

In [ ]:
!pip install mglearn

In [ ]:
!pip install --upgrade joblib==1.1.1

In [ ]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

## Aggregation des Trainingsdatenkorpus 

[Zelle # 1]

1-3: Die Variablen 'typNum', 'repert' und 'ausnahme' gelten für die Auswahl der im Trainingsdatenrepositorium vorhandenen Texte mit dem gesuchten Typ und dem Herkunftsrepertoire. Sowohl die Variable 'typNum' als auch 'repert' können mit leeren Anführungszeichen dargestellt werden. Für die Variable 'ausnahme' ist entweder ein unwahrscheinlicher Wert (z.B. 'xxyyzz') oder der Wert, mit dem eine bestimmte Gruppe von Daten von der Auswahl ausgeschlossen werden kann, verpflichtend.

4-8: Bei jeder Aggregation des Trainingsdatenkorpus wird der gleichnamige vorherige Korpus gelöscht.


In [ ]:
typNum='300'
repert=''
ausnahme='xxyyzz'
def entfernen(var):
    import os
    if os.path.exists(var):
        os.remove(var)
entfernen('episode_binar_train.csv')

[Zelle # 2]

1-59: Aggregation der Datei 'episode_temp_train.csv' (eine Funktion). Die im Ordner „Trainingsdaten“ angelegten TXT-Dateien werden in der CSV-Datei aggregiert und in das Root-Repository verschoben.

60-66: Umwandlung von 'episode_temp_train.csv' in die Datei 'episode_string_train.csv'.

67-72: Laden von 'episode_string_train.csv', Bereinigung von überflüssigen Zeichen und Umwandlung in das Trainingsdatenkorpus 'episode_binar_train.csv'.

ab 73: Unnötige Aggregationen werden gelöscht.

In [ ]:
def clean(line):
    line = line.replace("\n"," ").strip()
    line = line.replace("ä","ae").replace("ü","ue").replace("ö","oe")
    line = line.replace("ß","ss").replace(",","").replace("«","")
    line = line.replace("»","").replace(".","").replace(":","")
    line = line.replace(";","").replace('"',"")
    line = line.replace("?","").replace("!","").replace("á","a")
    line = line.replace(",","").replace("\t"," ").replace("'","")
    line = line.replace("‹","").replace("›","").replace("-"," ")
    line = line.replace("'('","").replace("')'","").replace('>','')
    line = line.replace("    "," ").replace("   "," ").replace("  "," ")
    line = line.replace('–','').replace('—','').replace('<','')
    line = line.replace("Â", "A").replace("ø", "oe").replace('“','')
    line = line.replace('„','').replace('(','').replace(')','')
    line = line.replace("‚", "").replace(']','').replace('[','')
    if line == "": 
        return
    line=line.split("|")
    line[0]=line[0].split("|")[0]
    for i, _ in enumerate(line):
        if (i !=0) and (i!=2):
            line[i]=line[i].lower()
    flex=[]
    try:
        flex=line[1].split("")
    except:
        pass
    value=str(line)
    line=str(line)   
    flex.append(line)
    ret=[]
    for i in flex:
        ret.append((i,value[0]))
    return ret
def write_back(words, temp):
    with open(temp, "a", encoding='utf-8') as output:      
        for word in words:
            as_lex = word[0]
            full_word = '"' + as_lex + '"'
            for sub_word in word[1:]:
                full_word += " , "  '"' + sub_word + '"'
            full_word +="\n"
            output.write(full_word)
def aggreg(temp, ordner):
    import glob
    with open(temp, "w", encoding='utf-8') as output:
        output.write ("quelle,episode,index_string,index_binar\n")
    sorted_files=sorted(glob.glob(ordner))
    pairs = []
    for file in sorted_files:
        if typNum in file and repert in file and not ausnahme in file:
            with open(file, 'r', encoding='utf-8', errors='ignore') as episode:
                for line in episode.readlines():
                    clean_words = clean(line)
                    pairs = pairs + clean_words
    write_back(pairs, temp)
temp = "episode_temp_train.csv"
ordner = "Trainingsdaten/*.txt"
aggreg(temp, ordner)
fin = open('episode_temp_train.csv','r', encoding ='utf-8')
fout = open('episode_string_train.csv', "wt", encoding ='utf-8')
for kfz in fin:
    fout.write(kfz.replace(', "["',"").replace('"[', "").replace(']"',"")
               .replace("', '", "','").replace(" '", "'"))
fin.close()
fout.close()
fin = open('episode_string_train.csv','r', encoding='utf-8') 
fout = open('episode_binar_train.csv', "wt", encoding='utf-8')
for efz in fin:
    fout.write(efz.replace("'",""))             
fin.close()
fout.close()
entfernen('episode_temp_train.csv')
entfernen('episode_string_train.csv')

##  Herstellung des Modells und der prototypischen Metaepisode 

[Zelle # 3]

1-5: Zugriff auf den Trainingsdatenkorpus nehmen und die im Datenfeld 'index_string' verzeichneten Merkmale sowie deren Häufigkeit anzeigen.

In [ ]:
import pandas as pd
from collections import Counter
df = pd.read_csv('episode_binar_train.csv',  encoding='utf-8')
indexliste=Counter(df.index_string)
print(indexliste, sep='\n')

[Zelle # 4]

1: Eeine Episode Kopieren und sie als Wert der Variable 'gesep' hier einfügen. 

In [ ]:
gesep = 'e300_c_anfangssituation_ankunft_und_erkundigung_der_not'

[Zelle # 5]

1-6: In den Trainingsdaten die gesuchte Episode ('gesep') mit Einsen (1) und den Rest mit Nullen (0) kennzeichnen, und die Änderungen speichern.

In [ ]:
for i, gesuchte_episode in enumerate(df.index_string):
    if (gesuchte_episode == gesep):
        df.index_binar[i]='1'
    else: 
            df.index_binar[i]='0'
df.to_csv('episode_binar_train.csv', encoding='utf-8', index=False)

[Zelle # 6]

1-4: Im Trainingsdatenkorpus die Datenfelder für Probetexte (X_train) und für die Merkmale (y_train) bestimmen und das Gleichgewicht zwischen negativ (0) und positiv (1) etikettierten Datensätzen analysieren (siehe Counter).

In [ ]:
X_train=(df['episode'])
y_train=(df['index_binar'])
indexliste=Counter(df.index_binar)
print(indexliste, sep='\n')

[Zelle # 7]

1: Die in die eckigen Klammern eingetragenen Wörter werden zusätzlich in die vorhandene Stopwörter-Liste eingefügt. Einzelne Wörter müssen dabei mit Anführungszeichen umschlossen und von anderen Wörtern durch Kommas getrennt werden, wie zum Beispiel: ['mein', 'dein'].

2-6: Das Datenfeld 'episode' von konventionellen und von uns eingetragenen Stoppwörtern bereinigen.

In [ ]:
custom_stop_word_list=[]
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
NLTK_stop_words_list=stopwords.words('german')
de = custom_stop_word_list + NLTK_stop_words_list

[Zelle # 8]

1-8: Einrichtung einer sciki-learn Pipeline mit Klassen LogisticRegression, TfidVectorizer und GridSearchCV. 

9-10: Modell auswerten (s. Bester Score aus der Kreuzvalidierung:).

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
pipe = make_pipeline(TfidfVectorizer(min_df=3, stop_words=de, ngram_range=(1,1), norm=None), 
                     LogisticRegression())
param_grid = {'logisticregression__C': [0.001, 0.01, 0.1, 1, 10]}     
grid = GridSearchCV(pipe, param_grid, cv=5)
grid.fit(X_train, y_train)
print("Bester Score aus der Kreuzvalidierung: {:.2f}".format(grid.best_score_))

[Zelle # 9]

1-6: Aus dem Modell die Liste der positiven und negativen Merkmale extrahieren, die Anzahl der Merkmale anzeigen (siehe Gesamte Anzahl der Wörter:).

In [ ]:
import numpy as np
vectorizer = grid.best_estimator_.named_steps["tfidfvectorizer"]
max_value = vectorizer.transform(X_train).max(axis=0).toarray().ravel()
sorted_by_tfidf = max_value.argsort()
feature_names = np.array(vectorizer.get_feature_names_out())
print("Gesamte Anzahl der Wörter: {}".format(len(vectorizer.vocabulary_)))

[Zelle # 10]

1-6: Merkmale mit den höchsten Koeffizienten visualisieren (blau=positiv, rot=negativ). Die Variable 's_zahl' legt fest, wie viele Merkmale beider Kategorien eingeblendet werden sollen.

7: Gibt den genauen Wert des kleinsten und höchsten Koeffizienten zurück.

In [ ]:
s_zahl = 200
import matplotlib.pyplot as plt
import mglearn
mglearn.tools.visualize_coefficients(
    grid.best_estimator_.named_steps["logisticregression"].coef_, 
    feature_names, n_top_features=s_zahl)
plt.grid()
#plt.loglog()
plt.ylim()
#plt.savefig(gesep+'.jpg', dpi=80)

[Zelle # 11a]

1-7: Extrahieren der Top-Klassifikationsmerkmale mit positiven und negativen Werten und ihrer Koeffizienten.

8-11: Visualisierung der ausgewählten Merkmale bzw. Koeffizienten.

In [ ]:
gio=(grid.best_estimator_.named_steps["logisticregression"].coef_[0])
n_top=np.argsort(gio)
z_top=np.sort(gio)
text_pos=feature_names[n_top[-s_zahl:]]
text_neg=feature_names[n_top[0:s_zahl]]
zahl_pos=gio[n_top[-s_zahl:]]
zahl_neg=gio[n_top[0:s_zahl]]
#print("Positive Merkmale:\n{}".format(text_pos))
#print ("Negative Merkmale:\n{}".format(text_neg))
#print ("Positive coef:\n{}".format(zahl_pos))
#print ("Negative coef:\n{}".format(zahl_neg))

 [Zelle # 11b]
 
1-7: Berechnung des Schwellenwerts nach der Gleichung (y, x) + (-y, -x). Wenn die Gleichung den Wert 0 ergibt, ist der Schwellenwert gleich x.
 
 

In [ ]:
kfz=range(s_zahl)
folgenum=0
for x in kfz:
    folgenum+=1
    vollliste=(zahl_pos[s_zahl-folgenum])+(zahl_neg[s_zahl-folgenum])
    rundliste=(round(vollliste,3))
    print(rundliste, '-', x)

[Zelle # 11c]

Schwellenwert für die Variable 'sw' manuel eintragen.

In [ ]:
sw=123

[Zelle # 12]

1-17: Bestimmte Merkmale in die neue txt Datei speichern(1-7); Gesamte Information durch die Datenfelder strukturieren (8-12) und nach dem Löschen der alten Datei (13) neue in Messdatenordner anlegen (14-17).

In [ ]:
import csv 
import shutil, os
filename = '0_'+gesep
tit=(text_pos[s_zahl-sw:s_zahl])
with open (filename+'.txt', 'wt', encoding='utf-8') as f:
    writer = csv.writer(f, delimiter=' ')
    writer.writerow(tit)
with open(filename+'.txt', 'r') as file:
    file_contents = file.read().replace("\n", "")
    frt_contents = filename+'|'+file_contents+'|0|0'
with open(filename+'.txt', 'wt', encoding='utf-8') as f:
        f.write(frt_contents)
entfernen('Messdaten/'+filename+'.txt')
import glob
files = glob.glob(filename+'.txt')
for f in files:
    shutil.move(f, 'Messdaten/')

##  Herstellung des Messdatenkorpus 

[Zelle # 13]

1-3: Die Variablen 'typNum', 'repert' und 'ausnahme' gelten für die Auswahl der im Trainingsdatenrepositorium vorhandenen Texte mit dem gesuchten Typ und dem Herkunftsrepertoire. Sowohl die Variable 'typNum' als auch 'repert' können mit leeren Anführungszeichen dargestellt werden. Für die Variable 'ausnahme' ist entweder ein unwahrscheinlicher Wert (z.B. 'xxyyzz') oder der Wert, mit dem eine bestimmte Gruppe von Daten von der Auswahl ausgeschlossen werden kann, verpflichtend.

4: Den vorhandenen Messdatenkorpus löschen (falls vorhanden).

5-7: Im Messdatenordner die Messdaten parsen und einen Korpus 'episode_temp_mess.csv' aggregieren.

8-16: 'episode_temp_mess.csv' von überflüssigen Zeichen bereinigen und in den Messdatenkorpus 'episode_binar_mess.csv' umwandeln.

17-21: Datenfelder deklarieren, Datensätze zählen (siehe Counter), anschließend unnötige temporäre Aggregationen löschen.

22-29: Den Messdatenkorpus aus den Episoden bestimmen und alle anderen Datenfelder ignorieren.

In [ ]:
typNum=''
repert=''
ausnahme='xxyyzz'
entfernen('episode_binar_mess.csv')
temp = "episode_temp_mess.csv"
ordner = "Messdaten/*.txt"
aggreg(temp, ordner)
from collections import Counter
fin = open("episode_temp_mess.csv",'r', encoding ='utf-8')
fout = open('episode_binar_mess.csv', "wt", encoding ='utf-8')
for efz in fin:
    fout.write(efz.replace(', "["',"").replace('"[', "")
               .replace(']"',"").replace("', '", "','")
               .replace(" '", "'").replace("'",""))   
fin.close()
fout.close()
df = pd.read_csv('episode_binar_mess.csv', encoding='utf-8')
from collections import Counter
indexliste=Counter(df.index_binar)
print(indexliste)
entfernen('episode_temp_mess.csv')
t_corpus = []
file = 'episode_binar_mess.csv'
with open(file, "r", encoding='utf-8') as maerchen:
    maerchen = df.episode
    reader = csv.reader(maerchen, delimiter = "|") 
    for row in reader:       
        lst = str(row)
        t_corpus.append(lst)

## Vorhersage von Logistic Regression

[Zelle # 14]

1-2: Vorhersagen treffen.

3: Der synthetische Datensatz wird ausgeschlossen.

4: Jeder positiv bewertete Datensatz wird separat eingeblendet.

In [ ]:
mod_pred=grid.predict(t_corpus)
for index in enumerate(mod_pred): 
    if '1' in (index) and index[0]>0:   
        print ('Positiv bewerteter Datensatz:{}'.format(index))

[Zelle # 15]

1-7: Alle probabilistischen Vorhersagen abrufen. Durch die Anpassung der Variable 'pos>=' in Zeile 6 kann der minimale Score manipuliert werden.

In [ ]:
all_pred=grid.predict_proba(t_corpus) 
sort_index=np.flip(all_pred[:,1].argsort())
for num_pos in sort_index:
    pos=all_pred[num_pos][1]
    for i,t_corpus_element in enumerate(t_corpus):
        if i==num_pos  and num_pos>0 and pos>0.007:
            print(num_pos,'-', pos,'-', df.quelle[num_pos],'-', df.episode[num_pos])

##  Vorhersage von Cosine Similarity

[Zelle # 16]

1-14: Daten im Messdatenkorpus Vektorisieren, Transformieren und die Metrik für die Vorhersagefunktion definieren.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
vectorizer = TfidfVectorizer
vectorizers = vectorizer(analyzer='word', ngram_range=(1,1),
                         min_df = 2, stop_words = de)
vokabular =  vectorizers.fit_transform (t_corpus)
metrik = cosine_similarity
def find_similar(vokabular, index, top_n =-1):   
    metriks = metrik(vokabular[index:index+1], vokabular).flatten()
    related_docs_indices = [i for i in metriks.argsort()[::-1] 
                            if i != index]
    return [(index, metriks[index]) for index 
            in related_docs_indices][0:top_n]

[Zelle # 17]

1-3: Gesuchte Episode (Zielepisode) einblenden. Die gesuchte Episode wird durch die 'n_te'-Variable bestimmt. Manuell eingepflegte Zielepisoden sollten deshalb immer in der ersten Reihe der Korpusdatensätze stehen. Wir empfehlen daher, diese Daten mit dem Präfix '0_' zu benennen.

4: Synthetische Datei aus dem Messdatenordner entfernen.

In [ ]:
n_te = 0
synt = t_corpus[n_te]
print(df.quelle[n_te],'-', synt)
entfernen('Messdaten/'+'0_'+gesep+'.txt')

[Zelle # 18]

1-3: Suchen starten und die Ergebnisse einblenden.

In [ ]:
for index, score in (find_similar(vokabular, n_te)):
    if (df.index_string[index]==0 and score>0.007):
        print(index,'-', score,'-', df.quelle[index],'-', 
           t_corpus[index], '-',  df.index_binar[index],'-', df.index_binar[index])